# NASAaccess Demonstration for the La Plata River Basin
##### Ibrahim Mohammed <ibrahim.mohammed@nasa.gov>

## Part 1: Exploring R language functionality

##### Demonstrating graphics capabilities:

In [ ]:
demo(graphics)

##### Plotting data:

In [ ]:
library(stats)
x<-1:10
y<-rnorm(10) # normal distribution with mean of 0 and standard deviation of 1
plot(x,y)

##### Performing arithmetic:

In [ ]:
3*(5+7.3)

##### Assigning and working with variables:

In [ ]:
g<-9.81
m<-30
f<-m*g
f

##### Combining values into a vector:

In [ ]:
a<-c(1,2,3,4)
b<-a*2
b

##### Common operations:

In [ ]:
ls()  # list the objects in your workspace
rm(f)   # Remove one object
rm(list=ls()) # Remove objects (to clean up)

##### Creating your own function:

In [ ]:
# Function to calculate harmonic mean
Harmonic_mean.r=function(x)
      {
          x<-x[!is.na(x)]
          a<-sum(1/x)
          n<-length(x)
          
          return((a/n)^-1)
              }

##### Try out the function:

In [ ]:
Harmonic_mean.r(x=c(1,2,4,NA))

##### Quiz: Testing your skills

Can you develop a function to calculate the geometric mean following the example shown above for the harmonic mean?

(Hint:The geometric mean is defined as the nth root of the product of n numbers, i.e., for a set of numbers x1, x2, ..., xn,)

## Part 2: Exploring hydrological data

### Precipitation analysis for Ruta Provincial, Argentina

Precipitation data retrieved from <https://apps.geoglows.org/apps/water-data-explorer>

Station/Platform Name: basavilbaso
Territory of origin of data: Argentina
Supervising Organization: Instituto Nacional del Agua (INA)
Geospatial Location: lat: 32°22'23'' lon: 58°53'12''

In [ ]:
# The file 4E47D870E717581F520F6C4EBE8E23962A880107_1.csv has been formatted as follows
# day,Precipitation (mm)
# 2008-12-29 09:00:00,0.2
# 2008-12-30 09:00:00,0
# 2008-12-31 09:00:00,0
# 2017-05-01 09:00:00,0
# 2017-05-02 09:00:00,0
# 2017-05-03 09:00:00,0
# 2017-05-04 09:00:00,0
# 2017-05-05 09:00:00,0
# 2017-05-06 09:00:00,0
# 2017-05-07 09:00:00,2

##### Read in data file:

In [ ]:
pp<-read.csv('./extdata/precip_data.csv',na.strings = '-9999',colClasses = c('POSIXct','numeric'))

##### Print summary of precipitation data:

In [ ]:
summary(pp)

##### Plot the data:

In [ ]:
plot(pp[,1],pp[,2],type="l",xlab='',ylab='(mm)',main='Rainfall at Basavilbaso, Argentina')

##### Print mean and variance of precipitation:

In [ ]:
# Mean
mean(pp[,2])

# Variance
var(pp[,2])

##### Create plot and save as image:

In [ ]:
dir.create('./Plots/')
jpeg(file='./Plots/Basavilbaso.jpeg',res = 300 ,units = 'in',width = 6, height = 5)
par(mar=c(4,5,4,1))
plot(pp[,1],pp[,2],type="h",xlab='',ylab='(mm)',
     main='Rainfall at Basavilbaso, Argentina',xlim = c(as.POSIXct('2018-1-1'),as.POSIXct('2021-10-1')),
     cex.lab=2,cex.axis=2)
abline(h=mean(pp[,2]),lwd=2,col=2)
box(lwd=3)
dev.off()

### Streamflow analysis for Ruta Provincial, Argentina

Station/Platform Name: Ruta Provincial nº 39
Territory of origin of data: Argentina
Supervising Organization: Instituto Nacional del Agua (INA)
Geospatial Location: lat: 32°26'42'' lon: 58°33'17''

##### Read in streamflow data:

In [ ]:
streamflow<-read.csv('./extdata/streamflow_data.csv',colClasses = c('POSIXct','numeric'))

##### Print the first several rows:

In [ ]:
head(streamflow)

In [ ]:
summary(streamflow)

##### Create streamflow time series

In [ ]:
library(xts, warn.conflicts = F, quietly = T)

streamflowseries<-xts(streamflow[,2],order.by = streamflow[,1])
annualstreamflow<-apply.yearly(streamflowseries,FUN=mean)
par(mar=c(4,6,4,1))
plot(index(annualstreamflow),coredata(annualstreamflow),'b',col='red',xlab = '',ylab = '',cex.axis=2,lwd=3,main='Annual Average Streamflow at Ruta Provincial, Argentina')
mtext(side=2,adj=0.5,text=expression(m^3/sec),cex=3,line=3.5)
box(lwd=4)

##### Save plot as an image:

In [ ]:
jpeg(file='./Plots/Ruta Provincial streamflow.jpeg',res = 300 ,units = 'in',width = 8, height = 6)
par(mar=c(4,7,4,1))
plot(index(annualstreamflow),coredata(annualstreamflow),'b',col='blue',xlab = '',ylab = '',cex.axis=2,lwd=3,main='Annual Average Streamflow at Ruta Provincial, Argentina')
mtext(side=2,adj=0.5,text=expression(m^3/sec),cex=3,line=3.5)
box(lwd=4)
dev.off()

### Watershed analysis

##### Load required packages:

In [ ]:
library(rgdal)
library(ncdf4)

##### Read in shapefile and create polygons

In [ ]:
watershed <- './extdata/cuenca del plata geog.shp'

polys <- readOGR(dsn=watershed,verbose = F)
summary(polys)

##### Plot the shapefile:

In [ ]:
plot(polys,col='red',main='LaPlata River Basin')
box(lwd=3)

##### Read in GPM (precipitation) data as a NetCDF:

In [ ]:
nc<-nc_open( './extdata/3B-DAY.MS.MRG.3IMERG.20160131-S000000-E235959.V03.nc4' )
print(nc)

##### Summarize file characteristics:

In [ ]:
# looking at file dimensions
summary(nc$dim)

# Extracting the y-values (longitudes in degrees east)
dim1<-nc$dim[[1]]
head(dim1)

nc.long<-ncvar_get(nc,dim1$name)
head(nc.long)

In [ ]:
# Extracting the x-values (latitudes in degrees north)
dim2<-nc$dim[[2]]

nc.lat<-ncvar_get(nc,dim2$name)

head(nc.lat)

##### Obtain precipitation values:

In [ ]:
v1<-nc$var[[1]]
v1$name

In [ ]:
ncatt_get(nc,varid='precipitationCal')

data<-ncvar_get(nc,v1)

##### Aligning the data to match long/lat:

In [ ]:
data<-data[ nrow(data):1, ]
nc_close(nc)

##### Plotting the precipitation data:

In [ ]:
library(raster)
library(maps)

In [ ]:
IMERG<-raster(x=as.matrix(data),xmn=nc.long[1],xmx=nc.long[NROW(nc.long)],ymn=nc.lat[1],ymx=nc.lat[NROW(nc.lat)],crs=CRS('+proj=longlat +datum=WGS84'))

# Plotting the climate data obtained from NetCdf
plot(IMERG,axes=F,main='Daily accumulated precipitation on 2016-01-31 in mm')
map(add=T)
map.axes(cex.axis=0.8)

##### Try plotting with Google Maps

In [ ]:
library(RgoogleMaps)

In [ ]:
lat <- range(nc.lat) #define our map's ylim
lon <- range(nc.long) #define our map's xlim
center <- c(-35.6, -58.4)  #tell what point to center on
zoom <- 5  #zoom: 1 = furthest out (entire globe), larger numbers = closer in
terrmap <- GetMap(center=center, zoom=zoom, maptype= "terrain", destfile = "./Plots/terrain.png", NEWMAP = TRUE) #lots of visual options, just like google maps: maptype = c("roadmap", "mobile", "satellite", "terrain", "hybrid", "mapmaker-roadmap", "mapmaker-hybrid")

##### Reading the study Basin DEM

In [ ]:
watershed.dem <- './extdata/la_plata.tif'
dem <- raster(watershed.dem)

##### Create a plot of our DEM raster along with Basin

In [ ]:
library(ggplot2)

In [ ]:
shape <- readOGR(watershed)
plot(dem,
     main="LaPlata Basin with Digital Elevation Model (DEM)", 
      col=rev(bpy.colors()), 
      xlab='lon', 
      ylab='lat',
      legend  = T,
      legend.args=list(text='Elevation (m)', 
                       side=4, 
                       font=2, 
                       line=2.5, 
                       cex=0.8))

plot(shape , add = TRUE)

## Part 3: Working with NASAaccess

In [ ]:
library(NASAaccess)

In [ ]:
# Look at help file for GPMswat
help(GPMswat)

##### Working with sample GPM data:

In [ ]:
GPM_NRT(Dir = "./INPUT/",
        watershed = watershed,
        DEM = watershed.dem,
        start = "2022-10-31",
        end = "2022-11-6")

***
Examine data at "./INPUT/"


What can you say about rainfall during the first 7 days of November over the LaPlata basin?
***

##### Looking at CMIP5 data:

In [ ]:
NEX_GDDP_CMIP5(
   Dir = "./INPUT2/",
   watershed = watershed,
   DEM = watershed.dem,
   start = "2060-12-1",
   end = "2060-12-10",
   model = "GFDL-CM3",
   type = "tas",
   slice = "rcp85")

Now examine data at "./INPUT2/"

##### Try another function from NASAaccess! 

##### Other examples are available at https://imohamme.github.io/NASAaccess/index.html 

### Thanks!